In [1]:
import tkinter as tk

In [2]:
class NaryNode:
    indent = '  '
    box_half_width = 80 / 2
    box_half_height = 40 / 2
    indent_leaf = 20
    x_spacing = 20
    y_spacing = 20

    def __init__(self, value=''):
        self.value = value
        self.children = []
        
        # Initialise drawing parameters
        self.centre = (0, 0)
        self.subtree_bounds = (
            self.centre[0] - self.box_half_width,
            self.centre[1] - self.box_half_height,
            self.centre[0] + self.box_half_width,
            self.centre[1] + self.box_half_height
        )

    def __str__(self, level=0):
        return (
            f"{self._indent_node(self.value, level=level)}:\n"
            f"{''.join([child.__str__(level=level+1) for child in self.children])}"
        )

    @classmethod
    def _indent_node(cls, value='', level=0):
        """
        Prepares a string indented to reflect the level of the empty child of the current node. 
        :param value: the text, if available, to include after the indentation 
        :param level: an integer to reflect the level (or depth) in the tree and the indentation required for the node
        :return: string containing enough indentation chars (usually spaces) followed by a text value if available
        """
        return "".join([cls.indent for _ in range(0, level)]) + value

    def add_child(self, child):
        self.children.append(child)

    def is_leaf(self):
        return not self.children
        
    def is_twig(self):
        return self.children and all([c.is_leaf() for c in self.children])
        
    def find_node(self, value):
        """
        Return the (first) node that has the corresponding value, starting from the current node
        and then traversing child nodes.
        :param value: the value to compare against each node
        :return: the node that matches the given value
        """
        if self.value == value:
            return self
        
        for child in self.children:
            node = child.find_node(value) if child else None
            if node:
                return node
        return None
    
    def traverse_preorder(self):
        nodes = [self]
        for c in self.children:
            nodes.extend(c.traverse_preorder())
        return nodes
    
    def traverse_postorder(self):
        nodes = []
        for c in self.children:
            nodes.extend(c.traverse_postorder())
        nodes.extend([self])
        return nodes
    
    def traverse_breadth_first(self):
        result = []
        queue = [self]
        
        while queue:
            node = queue.pop(0)
            result.append(node)
            
            for c in node.children:
                queue.append(c)
                
        return result
    
    def arrange_subtree(self, xmin, ymin):
        """Position the node's subtree"""
        # Calculate cy, the Y coordinate for this node.
        cy = ymin + self.box_half_height
        
        # If the node has no children, just place it here and return
        if self.is_leaf():
            self.centre = (xmin + self.box_half_width, cy)
            self.subtree_bounds = (
                xmin,
                ymin,
                self.centre[0] + self.box_half_width,
                self.centre[1] + self.box_half_height
            )
            return
        
        # Set child_xmin and child_ymin to the
        # start position for child subtrees
        child_xmin = xmin
        if self.is_twig():
            child_xmin += self.indent_leaf
        child_ymin = cy + self.box_half_height + self.y_spacing
        
        # Set ymax equal to the largest Y position used
        ymax = ymin + 2 * self.box_half_height
        
        # Position the child subtrees
        for child in self.children:
            # Position this child subtree
            child.arrange_subtree(child_xmin, child_ymin)
    
            # Update child_xmin to allow room for the subtree
            # and space between the subtrees
            if not self.is_twig():
                child_xmin = child.subtree_bounds[2] + self.x_spacing
            
            if self.is_twig():
                child_ymin = child.subtree_bounds[3] + self.y_spacing
    
            # Update the subtree bottom ymax
            ymax = max(ymax, child.subtree_bounds[3])
    
        # Set xmax equal to child_xmin minus the horizontal
        # spacing we added after the last subtree
        if self.is_twig():
            xmax = child_xmin + 2 * self.box_half_width
        else:
            xmax = child_xmin - self.x_spacing
    
        # Use xmin, ymin, xmax, ymax to set our subtree bounds
        self.subtree_bounds = (
            xmin,
            ymin,
            xmax,
            ymax
        )
    
        # Centre this node over the subtree bounds
        if self.is_twig():
            self.centre = (xmin + self.box_half_width, ymin + self.box_half_height)
        else:
            self.centre = ((xmin + xmax) / 2, ymin + self.box_half_height)
    
    def draw_subtree_links(self, canvas):
        """Draw the subtree's links"""
        if self.is_twig():
            verx = self.centre[0] - self.box_half_width + (self.indent_leaf / 2)
            canvas.create_line(
                verx,
                self.centre[1],
                verx,
                self.children[-1].centre[1]
            )
        # If we have exactly one child, just draw to it
        elif len(self.children) == 1:
            canvas.create_line(self.centre, self.children[0].centre, fill='green')
        
        # Else if we have more than one child
        # draw vertical and horizontal branches
        elif len(self.children) > 0:
            # calculate the Y coordinate for the horizontal line
            hy = self.centre[1] + self.box_half_height + (self.y_spacing / 2)
            # draw vertical line from this node to horizontal line
            canvas.create_line(self.centre, self.centre[0], hy, fill='green')
            # draw horizontal line
            canvas.create_line(
                self.children[0].centre[0],
                hy,
                self.children[-1].centre[0],
                hy,
                fill='green'
            )
        
        # Recursively draw child subtree links
        for child in self.children:
            if child.is_leaf():
                # draw horizontal line back to vertical line (coming from parent node)
                canvas.create_line(
                    child.centre,
                    child.centre[0] - self.box_half_width - (self.indent_leaf / 2),
                    child.centre[1],
                    fill='green'
                )
            else:
                # draw vertical line to horizontal line (above each child)
                canvas.create_line(
                    child.centre,
                    child.centre[0],
                    child.centre[1] - self.box_half_height - (self.y_spacing / 2),
                    fill='green'
                )

            child.draw_subtree_links(canvas)
        
        # Outline the subtree for debugging
        #canvas.create_rectangle(self.subtree_bounds, fill='', outline='red')
        
    def draw_subtree_nodes(self, canvas):
        """Draw the subtree's nodes"""
        # Draw the node
        canvas.create_rectangle(
            self.centre[0] - self.box_half_width, 
            self.centre[1] - self.box_half_height, 
            self.centre[0] + self.box_half_width, 
            self.centre[1] + self.box_half_height, 
            fill='white' if self.is_leaf() else 'pink'
        )
        canvas.create_text(self.centre, text=self.value, fill='red')
        
        # Draw the descendants' nodes
        for child in self.children:
            child.draw_subtree_nodes(canvas)
        
    def arrange_and_draw_subtree(self, canvas, xmin, ymin):
        # Position the tree
        self.arrange_subtree(xmin, ymin)
        
        # Draw the links
        self.draw_subtree_links(canvas)
        
        # Draw the nodes
        self.draw_subtree_nodes(canvas)


In [3]:
# Build a test org chart.
#                      GeneriGloop
#                           |
#     +-------------+-------+---------------------------+
#    R&D          Sales                       Professional Services
#    |            |                                     |
#    |            |                      +--------------+---------------+
#    +- Applied   +- Inside Sales        HR             Accounting    Legal
#    |            |                      |              |             |
#    +- Basic     +- Outside Sales       +- Training    +- Payroll    +- Compliance
#    |            |                      |              |             |
#    +- Advanced  +- B2B                 +- Hiring      +- Billing    +- Progress Prevention
#    |            |                      |              |             |
#    +- Sci Fi    +- Consumer            +- Equity      +- Reporting  +- Bail Services
#                 |                      |              |
#                 +- Account Management  +- Discipline  +- Opacity
#
root = NaryNode('GeneriGloop')
rd = NaryNode('R & D')
sales = NaryNode('Sales')
pf = NaryNode('Professional\n    Services')
hr = NaryNode('HR')
accounting = NaryNode('Accounting')
legal = NaryNode('Legal')

applied = NaryNode('Applied')
basic = NaryNode('Basic')
advanced = NaryNode('Advanced')
scifi = NaryNode('Sci Fi')

inside_sales = NaryNode('Inside Sales')
outside_sales = NaryNode('Outside Sales')
b2b = NaryNode('B2B')
consumer = NaryNode('Consumer')
acc_mgmt = NaryNode('     Account\n Management')

training = NaryNode('Training')
hiring = NaryNode('Hiring')
equity = NaryNode('Equity')
discipline = NaryNode('Discipline')

payroll = NaryNode('Payroll')
billing = NaryNode('Billing')
reporting = NaryNode('Reporting')
opacity = NaryNode('Opacity')

compliance = NaryNode('Compliance')
progress_prevention = NaryNode('  Progress\nPrevention')
bail_services = NaryNode('   Bail\nServices')

root.add_child(rd)
root.add_child(sales)
root.add_child(pf)

rd.add_child(applied)
rd.add_child(basic)
rd.add_child(advanced)
rd.add_child(scifi)

sales.add_child(inside_sales)
sales.add_child(outside_sales)
sales.add_child(b2b)
sales.add_child(consumer)
sales.add_child(acc_mgmt)

pf.add_child(hr)
pf.add_child(accounting)
pf.add_child(legal)

hr.add_child(training)
hr.add_child(hiring)
hr.add_child(equity)
hr.add_child(discipline)

accounting.add_child(payroll)
accounting.add_child(billing)
accounting.add_child(reporting)
accounting.add_child(opacity)

legal.add_child(compliance)
legal.add_child(progress_prevention)
legal.add_child(bail_services)


In [4]:
def kill_callback():
    """A callback to destroy the tkinter window."""
    window.destroy()


In [5]:
window = tk.Tk()
window.title('org_chart1')
window.protocol('WM_DELETE_WINDOW', kill_callback)
window.geometry('620x440')

canvas = tk.Canvas(window, bg='white', borderwidth=2, relief=tk.SUNKEN)
canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Draw the tree
root.arrange_and_draw_subtree(canvas, 10, 10)

window.focus_force()
window.mainloop()